In [2]:
import pandas as pd

import numpy as np


In [5]:
# Read the CSV file
df_idp_cleaned = pd.read_csv("/Users/bercinersoz/Desktop/HSOS_idp_cleaned.csv", delimiter=",", low_memory=False)


In [6]:
df_idp_cleaned

,Year,Month,Governorate,Governorate p-code,District,District p-code,Sub-district,Sub-district p-code,Community,Community p-code,...,What was the third top priority need for the IDP population in the assessed location?_Infrustructure,What was the third top priority need for the IDP population in the assessed location?_Livelihoods,What was the third top priority need for the IDP population in the assessed location?_Money / Cash,What was the third top priority need for the IDP population in the assessed location?_NFIs,What was the third top priority need for the IDP population in the assessed location?_No Data,What was the third top priority need for the IDP population in the assessed location?_Protection,What was the third top priority need for the IDP population in the assessed location?_Shelter,What was the third top priority need for the IDP population in the assessed location?_Summer items,What was the third top priority need for the IDP population in the assessed location?_WASH,What was the third top priority need for the IDP population in the assessed location?_Winterisation
0,2021,1,Aleppo,SY02,Menbij,SY0205,Abu Qalqal,SY020501,Jeb Elkajli,C1804,...,0,1,0,0,0,0,0,0,0,0
1,2021,1,Aleppo,SY02,Menbij,SY0205,Abu Qalqal,SY020501,Jeifiyet Elmashi,C1807,...,0,1,0,0,0,0,0,0,0,0
2,2021,1,Aleppo,SY02,Menbij,SY0205,Abu Qalqal,SY020501,Jeb Eltawil,C1811,...,0,1,0,0,0,0,0,0,0,0
3,2021,1,Aleppo,SY02,Menbij,SY0205,Abu Qalqal,SY020501,Abu Qalqal,C1798,...,0,1,0,0,0,0,0,0,0,0
4,2021,1,Aleppo,SY02,Ain Al Arab,SY0206,Ain al Arab,SY020600,Ain Al Arab,C1946,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32907,2023,12,Aleppo,SY02,Jebel Saman,SY0200,Atareb,SY020001,Jeineh,C1032,...,0,0,0,0,0,0,0,0,0,1
32908,2023,12,Idleb,SY07,Harim,SY0703,Armanaz,SY070305,Armanaz,C4176,...,0,0,0,0,0,0,0,0,0,0
32909,2023,12,Idleb,SY07,Jisr-Ash-Shugur,SY0704,Jisr-Ash-Shugur,SY070400,Western Marj Akhdar,C4212,...,0,0,0,1,0,0,0,0,0,0
32910,2023,12,Idleb,SY07,Idleb,SY0700,Teftnaz,SY070004,Teftnaz,C3932,...,0,1,0,0,0,0,0,0,0,0


 # Merge with Exchange Rate

In [7]:
# Create a new column with the date format YYYY-MM-DD
df_idp_cleaned['Date'] = pd.to_datetime(df_idp_cleaned['Year'].astype(str) + '-' + df_idp_cleaned['Month'].astype(str) + '-01')

# Check the result
print(df_idp_cleaned[['Year', 'Month', 'Date']].head())

   Year  Month       Date
0  2021      1 2021-01-01
1  2021      1 2021-01-01
2  2021      1 2021-01-01
3  2021      1 2021-01-01
4  2021      1 2021-01-01


In [8]:
exchange_rates = pd.read_excel("/Users/bercinersoz/Desktop/Market/exchange_rates.xlsx")

In [9]:
exchange_rates.head()

,Date,TRY/USD,SYP/USD
0,2021-01-01,0.1366,0.0008
1,2021-02-01,0.1346,0.0008
2,2021-03-01,0.1212,0.0008
3,2021-04-01,0.1204,0.0004
4,2021-05-01,0.1177,0.0004


In [10]:
exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date'])

# Merge exchange rates with NWS and NES data
df_idp_cleaned = df_idp_cleaned.merge(exchange_rates[['Date', 'SYP/USD']], 
                                        left_on='Date', right_on='Date', how='left')

In [11]:
df_idp_cleaned.columns

Index(['Year', 'Month', 'Governorate', 'Governorate p-code', 'District',
       'District p-code', 'Sub-district', 'Sub-district p-code', 'Community',
       'Community p-code',
       ...
       'What was the third top priority need for the IDP population in the assessed location?_Money / Cash',
       'What was the third top priority need for the IDP population in the assessed location?_NFIs',
       'What was the third top priority need for the IDP population in the assessed location?_No Data',
       'What was the third top priority need for the IDP population in the assessed location?_Protection',
       'What was the third top priority need for the IDP population in the assessed location?_Shelter',
       'What was the third top priority need for the IDP population in the assessed location?_Summer  items',
       'What was the third top priority need for the IDP population in the assessed location?_WASH',
       'What was the third top priority need for the IDP population in the 

In [12]:
# Only numerics & > than 1
numeric_columns = df_idp_cleaned.select_dtypes(include=["number"])


columns_with_values_greater_than_one = numeric_columns.columns[(numeric_columns > 1).any()].tolist()


print(columns_with_values_greater_than_one)



['Year', 'Month', 'Proportion of IDPs living in solid finished apartments', 'Proportion of IDPs living in hotels', 'Proportion of IDPs living in unfinished or abandoned residential buildings', 'Proportion of IDPs living in damaged residential buildings', 'Proportion of IDPs living in tents', 'Proportion of IDPs living in collective center with more than one family', 'Proportion of IDPs living in concrete block shelters', 'Proportion of IDPs living in containers', 'Proportion of IDPs living in non residential structures with one family', 'Proportion of IDPs living in makeshift shelters with scavenged materials', "Proportion of IDPs living in 'other' types of shelter", 'Proportion of IDPs living in overcrowded shelters', 'Shelter occupancy situation - Proportion of IDPs that own their shelter', 'Shelter occupancy situation - Proportion of IDPs that rent their shelter', 'Shelter occupancy situation - Proportion of IDPs that are hosted for free', 'Shelter occupancy situation - Proportion o

In [13]:
df_idp_cleaned['SYP/USD'].isna().sum()

0

In [14]:
# Convert  data from SYP to USD
columns_to_convert = ['Average price of rent for a 2 bedroom apartment in the last 30 days', 
                      'Average monthly expenditure for water for a household of 6 people in the last 30 days', 
                      'Estimated daily wage for unskilled labour for IDPs in the assessed community']



for col in columns_to_convert:
    df_idp_cleaned[col] = df_idp_cleaned[col] * df_idp_cleaned['SYP/USD']


In [15]:
# Drop the 'Date', and 'SYP/USD' columns from combined_df
columns_to_drop = ['Date', 'SYP/USD']

df_idp_cleaned = df_idp_cleaned.drop(columns=columns_to_drop)

In [16]:
df_idp_cleaned.columns

Index(['Year', 'Month', 'Governorate', 'Governorate p-code', 'District',
       'District p-code', 'Sub-district', 'Sub-district p-code', 'Community',
       'Community p-code',
       ...
       'What was the third top priority need for the IDP population in the assessed location?_Infrustructure',
       'What was the third top priority need for the IDP population in the assessed location?_Livelihoods',
       'What was the third top priority need for the IDP population in the assessed location?_Money / Cash',
       'What was the third top priority need for the IDP population in the assessed location?_NFIs',
       'What was the third top priority need for the IDP population in the assessed location?_No Data',
       'What was the third top priority need for the IDP population in the assessed location?_Protection',
       'What was the third top priority need for the IDP population in the assessed location?_Shelter',
       'What was the third top priority need for the IDP population

# Market Data Merge

In [17]:
market_df = pd.read_excel("/Users/bercinersoz/Desktop/combined_df.xlsx")  

In [18]:
market_df.columns

Index(['month2', 'region', 'q_gov', 'admin1Name_en', 'q_district',
       'admin2Name_en', 'q_sbd', 'admin3Name_en', 'q_town', 'admin4Name_en',
       'smeb_complete', 'smeb_only_missing_water', 'Price_SMEB_total_wfloat',
       'Price_SMEB_total_sanswater', 'Price_smeb_incomplete', 'Price_SMEB_usd',
       'Price_SMEB_food', 'Price_SMEB_nfi', 'Price_SMEB_cookingfuels',
       'Price_SMEB_water', 'Price_SMEB_gbdata'],
      dtype='object')

In [19]:
# Grouping market data at a certain level
market_agg = market_df.groupby(['month2', 'admin1Name_en', 'admin3Name_en']).agg({
    'Price_SMEB_food': 'mean',   # Example: If there is price data, take its average
    'Price_SMEB_cookingfuels': 'mean',  
    'Price_SMEB_water': 'mean',  
    'Price_SMEB_gbdata': 'mean', 
}).reset_index()


print(market_agg.head())


      month2 admin1Name_en              admin3Name_en  Price_SMEB_food  \
0 2021-01-01    Al-Hasakeh                 Al-Hasakeh       174.666133   
1 2021-01-01    Al-Hasakeh              Al-Malikeyyeh       168.261800   
2 2021-01-01    Al-Hasakeh                      Amuda       223.800000   
3 2021-01-01    Al-Hasakeh                   Areesheh       172.383333   
4 2021-01-01    Al-Hasakeh  Be'r Al-Hulo Al-Wardeyyeh              NaN   

   Price_SMEB_cookingfuels  Price_SMEB_water  Price_SMEB_gbdata  
0                 6.413333            12.072           1.400000  
1                 1.973333               NaN           1.153333  
2                 2.000000               NaN           2.000000  
3                 7.333333            13.728           2.266667  
4                 7.000000            15.300           3.200000  


In [215]:
hsos_to_merge = df_idp_cleaned.copy()

In [221]:
from fuzzywuzzy import process

# Create Date for HSOS
hsos_to_merge['Date'] = pd.to_datetime(hsos_to_merge[['Year', 'Month']].assign(Day=1))


# First, let's merge with exact match
merged_df = pd.merge(hsos_to_merge, market_agg, 
                     left_on=['Date', 'Governorate', 'Sub-district'], 
                     right_on=['month2', 'admin1Name_en', 'admin3Name_en'], 
                     how='left')

from rapidfuzz import process

# Fuzzy matching 
def fuzzy_match(query, choices, threshold=80):
    match = process.extractOne(query, choices)
    if match and match[1] >= threshold:
        return match[0]
    return None

# Fill in missing matches with fuzzy matching
missing_rows = merged_df[merged_df['admin3Name_en'].isnull()]
if not missing_rows.empty:
    for idx, row in missing_rows.iterrows():
        matched_subdistrict = fuzzy_match(row['Sub-district'], market_agg['admin3Name_en'].tolist())
        if matched_subdistrict:
            merged_df.loc[idx, 'admin3Name_en'] = matched_subdistrict

# Recombine the results after fuzzy
merged_df = pd.merge(hsos_to_merge, market_agg, 
                     left_on=['Date', 'Governorate', 'Sub-district'], 
                     right_on=['month2', 'admin1Name_en', 'admin3Name_en'], 
                     how='left')



In [222]:
merged_df

,Year,Month,Governorate,Governorate p-code,District,District p-code,Sub-district,Sub-district p-code,Community,Community p-code,...,What was the third top priority need for the IDP population in the assessed location?_WASH,What was the third top priority need for the IDP population in the assessed location?_Winterisation,Date,month2,admin1Name_en,admin3Name_en,Price_SMEB_food,Price_SMEB_cookingfuels,Price_SMEB_water,Price_SMEB_gbdata
0,2021,1,Aleppo,SY02,Menbij,SY0205,Abu Qalqal,SY020501,Jeb Elkajli,C1804,...,0,0,2021-01-01,2021-01-01,Aleppo,Abu Qalqal,142.427200,2.500000,4.15800,4.825200
1,2021,1,Aleppo,SY02,Menbij,SY0205,Abu Qalqal,SY020501,Jeifiyet Elmashi,C1807,...,0,0,2021-01-01,2021-01-01,Aleppo,Abu Qalqal,142.427200,2.500000,4.15800,4.825200
2,2021,1,Aleppo,SY02,Menbij,SY0205,Abu Qalqal,SY020501,Jeb Eltawil,C1811,...,0,0,2021-01-01,2021-01-01,Aleppo,Abu Qalqal,142.427200,2.500000,4.15800,4.825200
3,2021,1,Aleppo,SY02,Menbij,SY0205,Abu Qalqal,SY020501,Abu Qalqal,C1798,...,0,0,2021-01-01,2021-01-01,Aleppo,Abu Qalqal,142.427200,2.500000,4.15800,4.825200
4,2021,1,Aleppo,SY02,Ain Al Arab,SY0206,Ain al Arab,SY020600,Ain Al Arab,C1946,...,0,1,2021-01-01,2021-01-01,Aleppo,Ain al Arab,158.660000,2.240000,5.40000,4.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32907,2023,12,Aleppo,SY02,Jebel Saman,SY0200,Atareb,SY020001,Jeineh,C1032,...,0,1,2023-12-01,2023-12-01,Aleppo,Atareb,104.433188,12.882000,4.57650,3.005235
32908,2023,12,Idleb,SY07,Harim,SY0703,Armanaz,SY070305,Armanaz,C4176,...,0,0,2023-12-01,2023-12-01,Idleb,Armanaz,108.484237,12.838777,4.57650,4.201905
32909,2023,12,Idleb,SY07,Jisr-Ash-Shugur,SY0704,Jisr-Ash-Shugur,SY070400,Western Marj Akhdar,C4212,...,0,0,2023-12-01,2023-12-01,Idleb,Jisr-Ash-Shugur,108.285075,12.824370,5.59350,2.959470
32910,2023,12,Idleb,SY07,Idleb,SY0700,Teftnaz,SY070004,Teftnaz,C3932,...,0,0,2023-12-01,2023-12-01,Idleb,Teftnaz,108.522375,12.824370,5.33925,4.407000


In [224]:
# Check the number of missing values
missing_counts = merged_df[['admin1Name_en', 'admin3Name_en', 'Price_SMEB_food', 
                           'Price_SMEB_cookingfuels', 'Price_SMEB_water', 
                           'Price_SMEB_gbdata']].isnull().sum()


print(missing_counts)


admin1Name_en              3469
admin3Name_en              3469
Price_SMEB_food            4756
Price_SMEB_cookingfuels    3669
Price_SMEB_water           7196
Price_SMEB_gbdata          3631
dtype: int64


In [226]:
# Checking random example in market
filtered_data = market_df[
    (market_df['admin3Name_en'] == "A'zaz") & 
    (market_df['month2'] == '2021-01-01')
]


print(filtered_data)


       month2     region q_gov admin1Name_en q_district admin2Name_en  \
62 2021-01-01  northwest  SY02        Aleppo     SY0204         A'zaz   
63 2021-01-01  northwest  SY02        Aleppo     SY0204         A'zaz   
64 2021-01-01  northwest  SY02        Aleppo     SY0204         A'zaz   
65 2021-01-01  northwest  SY02        Aleppo     SY0204         A'zaz   

       q_sbd admin3Name_en q_town admin4Name_en  ...  smeb_only_missing_water  \
62  SY020400         A'zaz  C1557      Shmarekh  ...                        0   
63  SY020400         A'zaz  C1561        Salama  ...                        0   
64  SY020400         A'zaz  C1562         Jarez  ...                        0   
65  SY020400         A'zaz  C1564          Azaz  ...                        0   

    Price_SMEB_total_wfloat  Price_SMEB_total_sanswater  \
62                      NaN                         NaN   
63                      NaN                         NaN   
64                      NaN                        

In [227]:
# # Checking random example in merged data
filtered_data_merged = merged_df[
    (merged_df['admin3Name_en'] == "A'zaz") & 
    (merged_df['month2'] == '2021-01-01')
]

print(filtered_data_merged)

      Year  Month Governorate Governorate p-code District District p-code  \
772   2021      1      Aleppo               SY02    A'zaz          SY0204   
773   2021      1      Aleppo               SY02    A'zaz          SY0204   
774   2021      1      Aleppo               SY02    A'zaz          SY0204   
775   2021      1      Aleppo               SY02    A'zaz          SY0204   
776   2021      1      Aleppo               SY02    A'zaz          SY0204   
777   2021      1      Aleppo               SY02    A'zaz          SY0204   
780   2021      1      Aleppo               SY02    A'zaz          SY0204   
781   2021      1      Aleppo               SY02    A'zaz          SY0204   
783   2021      1      Aleppo               SY02    A'zaz          SY0204   
784   2021      1      Aleppo               SY02    A'zaz          SY0204   
785   2021      1      Aleppo               SY02    A'zaz          SY0204   
787   2021      1      Aleppo               SY02    A'zaz          SY0204   

In [229]:
merged_df.columns

Index(['Year', 'Month', 'Governorate', 'Governorate p-code', 'District',
       'District p-code', 'Sub-district', 'Sub-district p-code', 'Community',
       'Community p-code',
       ...
       'What was the third top priority need for the IDP population in the assessed location?_WASH',
       'What was the third top priority need for the IDP population in the assessed location?_Winterisation',
       'Date', 'month2', 'admin1Name_en', 'admin3Name_en', 'Price_SMEB_food',
       'Price_SMEB_cookingfuels', 'Price_SMEB_water', 'Price_SMEB_gbdata'],
      dtype='object', length=989)

In [230]:
file_path = '/Users/bercinersoz/Desktop/HSOS_Market_merged_data.csv'

merged_df.to_csv(file_path, index=False)
print(f"File is saved: {file_path}")


File is saved: /Users/bercinersoz/Desktop/HSOS_Market_merged_data.csv


In [231]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32912 entries, 0 to 32911
Columns: 989 entries, Year to Price_SMEB_gbdata
dtypes: datetime64[ns](2), float64(879), int64(98), object(10)
memory usage: 248.3+ MB
